In [1]:
from BB_MesoWest.get_MesoWest import get_mesowest_ts, get_mesowest_stninfo

In [1]:
_service = {'auth', 'networks', 'networktypes', 'variables', 'qctypes'}
_station = {'metadata', 'timeseries', 'precipitation', 'nearesttime', 'latest'}
_service.update(_station)
_service, _station

({'auth',
  'latest',
  'metadata',
  'nearesttime',
  'networks',
  'networktypes',
  'precipitation',
  'qctypes',
  'timeseries',
  'variables'},
 {'latest', 'metadata', 'nearesttime', 'precipitation', 'timeseries'})

In [265]:
zz = {'START':1}
{k.lower(): v for k, v in zz.items()}

{'start': 1}

In [116]:
import sys
from datetime import datetime
import numpy as np
import requests
import urllib
import pandas as pd
from datetime import datetime

from get_credentials import get_MW_token
from BB_wx_calcs.wind import spddir_to_uv

# API Token
# Get your own token here: https://developers.synopticdata.com/
_token = get_MW_token()

# API Services
# https://developers.synopticdata.com/mesonet/v2/
_service = {'auth', 'networks', 'networktypes', 'variables', 'qctypes'}
_stations = {'metadata', 'timeseries', 'precipitation', 'nearesttime', 'latest'}
_service.update(_stations)

def synoptic_api(service, verbose=True, **params):
    '''
    Request data from the Synoptic API
    
    API References
    --------------
    - https://developers.synopticdata.com/mesonet/v2/
    - https://developers.synopticdata.com/mesonet/explorer/
    
    Parameters
    ----------
    service : str
        API service to use, including {'auth', 'latest', 'metadata',
        'nearesttime', 'networks', 'networktypes', 'precipitation',
        'qctypes', 'timeseries', 'variables'}
    params : keyword arguments
        API request parameters (arguments).
        May supply a list (i.e., stations) which will be converted to 
        the required comma separated list.
        Dates (i.e., start, end, obrange) may be given as a datetime
        or pandas.datetime (will be parsed by f-string to YYYYmmddHHMM).
    
    Returns
    -------
    A ``requests.models.Response`` object from ``requests.get(URL, params)``
    
    Examples
    --------
    To read the json data for metadata for a station
    
    >>> synoptic_api('metadata', stid='WBB').json()
    
    >>> synoptic_api('metadata', stid=['WBB', 'KSLC']).json()
    
    '''   
    help_url = 'https://developers.synopticdata.com/mesonet/v2/'
    assert service in _service, f"`service` must be one of {_service}. {help_url}"
    
    ## Service URL
    ##------------
    root = 'https://api.synopticdata.com/v2/'
    
    if service in _stations:    
        URL = f"{root}/stations/{service}"
    else:
        URL = f"{root}/{service}"
        
    ## Set API token
    ##--------------
    ## Default is set at top of this file, but may overwrite with keyword argument.
    params.setdefault('token', _token)
    
    ## Parse parameters
    ##-----------------
    # Change some keword parameters to the appropriate request format
    
    ## 1) Force all param keys to be lower case
    params = {k.lower(): v for k, v in params.items()}
    
    ## 1) Join lists to comma separtated strings.
    ##    For example, stid=['KSLC', 'KMRY'] --> stid='KSLC,KRMY').
    for key, value in params.items():           
        if isinstance(value, list) and key not in ['obrange']:
            params[key] = ','.join(value)
        
    ## 2) Datetimes should be string: 'YYYYmmddHHMM' or 'YYYYmmdd'
    if 'start' in params and not isinstance(params['start'], str):
        params['start'] = f"{params['start']:%Y%m%d%H%M}"
    if 'end' in params and not isinstance(params['end'], str):
        params['end'] = f"{params['end']:%Y%m%d%H%M}"
    if 'obrange' in params and not isinstance(params['obrange'], str): 
        # obrange should be a date or list of two dates.
        if not hasattr(params['obrange'], '__len__'):
            params['obrange'] = [params['obrange']]
        params['obrange'] = ','.join([f'{i:%Y%m%d}' for i in params['obrange']])    
    
    
    ##################
    # Make the request
    ##################
    f = requests.get(URL, params)
    
    # Check Data
    code = f.json()['SUMMARY']['RESPONSE_CODE']
    msg = f.json()['SUMMARY']['RESPONSE_MESSAGE']
    decode_url = urllib.parse.unquote(f.url)
    
    assert code == 1, f"🛑 There are errors in the API request {decode_url}. {msg}"
    
    if verbose:
        print(f'\n 🚚💨 Speedy Delivery from Synoptic API [{service}]: {decode_url}\n')
    
    return f

def stations_metadata(verbose=True, **params):
    """
    Get station metadata for stations as a Pandas DataFrame.

    https://developers.synopticdata.com/mesonet/v2/stations/metadata/

    Parameters
    ----------
    params : keyword arguments
        Synoptic API arguments used to specify the data request.
        
    Some useful arguments include:
    
    stid : str or list
        Station id or list of station ids
        ``['KSLC', 'UKBKB', 'KMRY']`` *or* ``'KSLC'``
    obrange : datetime or 2-item list of datetime
         Observation time range station must be active, as list of 
         datetimes.
    radius : str
        ``"lat,lon,miles"`` *or* ``"stid,miles"``
    state : str or list
        string or list of abbreviated state strings, i.e. ['UT','CA']
    vars : str or list
        Filter list of stations for those that report the listed variables.
        i.e., ``['air_temp', 'wind_speed', 'wind_direction', etc.]``
    varsoperator : {'and', 'or'}
    network - int
        Network integer number. (see network API service)
    """
    assert len(params) > 0, "🤔 Please assign a parameter (i.e., stid, radius, etc.)"

    # Get the data
    web = synoptic_api('metadata', verbose=verbose, **params)
    data = web.json()
        
    # Initialize a DataFrame
    df = pd.DataFrame(data['STATION'], index=[i['STID'] for i in data['STATION']])
    
    # Convert data to numeric values (if possible)
    df = df.apply(pd.to_numeric, errors='ignore')

    # Deal with "Period Of Record" dictionary
    df = pd.concat([df, df.PERIOD_OF_RECORD.apply(pd.Series)], axis=1)
    df[['start', 'end']] = df[['start', 'end']].apply(pd.to_datetime)

    # Rename some fields.
    # latitude and longitude are made lowercase to conform to CF standard
    df.drop(columns=['PERIOD_OF_RECORD'], inplace=True)
    df.rename(columns=dict(LATITUDE='latitude', LONGITUDE='longitude',
                           start='RECORD_START', end='RECORD_END'),
              inplace=True)
    
    df.attrs['URL'] = urllib.parse.unquote(web.url)
    df.attrs['elevation units'] = 'ft'
    df.attrs['SUMMARY'] = data['SUMMARY']
    return df.transpose().sort_index()

def stations_timeseries(verbose=True, **params):
    """
    Get station data for time series.

    https://developers.synopticdata.com/mesonet/v2/stations/timeseries/

    Parameters
    ----------
    params : keyword arguments
        Synoptic API arguments used to specify the data request.
        Must include 'start' and 'end' argument *or* 'recent'
    
    Some useful arguments include:
    
    start : datetime
        Start of the time series
    end : datetime
        End of the time series
    recent : int
        Instead of a start and end time, you can request values for 
        most recent X minutes.
    stid : str or list
        Station id or list of station ids
        ``['KSLC', 'UKBKB', 'KMRY']`` *or* ``'KSLC'``
    radius : str
        ``"lat,lon,miles"`` *or* ``"stid,miles"``
    state : str or list
        string or list of abbreviated state strings, i.e. ['UT','CA']
    vars : str or list
        Filter list of stations for those that report the listed variables.
        i.e., ``['air_temp', 'wind_speed', 'wind_direction', etc.]``
    varsoperator : {'and', 'or'}
    network - int
        Network integer number. (see network API service)
    units : {'metric', 'english'}
    obtimezone : {'UTC', 'local'}
    status : {'active', 'inactive'}
    
    Examples
    --------
    >>> stations_timeseries(stid='WBB', recent=100)
    >>> stations_timeseries(radius='UKBKB,10', vars='air_temp', recent=100)
    """
    assert len(params) > 0, "🤔 Please assign a parameter (i.e., stid, radius, etc.)"

    # Get the data
    web = synoptic_api('timeseries', verbose=verbose, **params)
    data = web.json()
    
    # Build a separate pandas.DataFrame for each station.
    Z = []
    for stn in data['STATION']:
        obs = stn.pop('OBSERVATIONS')
        senvars = stn.pop('SENSOR_VARIABLES')
        
        df = pd.DataFrame(obs).set_index('date_time')
        df.index = pd.to_datetime(df.index)
        
        # Break wind into U and V components, if speed and direction are available
        if all(['wind_speed' in senvars, 'wind_direction' in senvars]):
            for i_spd, i_dir in zip(senvars['wind_speed'].keys(),
                                    senvars['wind_direction'].keys()):
                u, v = spddir_to_uv(obs[i_spd], obs[i_dir])
                this_set = '_'.join(i_spd.split('_')[-2:])
                df[f'wind_u_{this_set}'] = u
                df[f'wind_v_{this_set}'] = v
        
        # Remove 'set_1d' and 'set_1d' from column names.
        # Sets 2+ will retain the full name. The user should refer to
        # the SENSOR_VARIABLES to see which are derived variables.
        col_names = {i: i.replace('_set_1d', '').replace('_set_1', '') for i in df.columns}
        df.rename(columns=col_names, inplace=True)
              
        # Remaining data in dict will be returned as attribute
        df.attrs = stn
        
        # Convert some strings to flaot/int
        for k, v in df.attrs.items():
            if isinstance(v, str):
                try:
                    n = float(v)
                    if n.is_integer():
                        df.attrs[k] = int(n)
                    else:
                        df.attrs[k] = n
                except:
                    pass
        
        # Rename lat/lon to lowercase to match CF convenctions
        df.attrs['latitude'] = df.attrs.pop('LATITUDE')
        df.attrs['longitude'] = df.attrs.pop('LONGITUDE')
        
        # Include other info
        df.attrs['UNITS'] = data['UNITS']
        df.attrs['QC_SUMMARY'] = data['QC_SUMMARY']
        df.attrs['SUMMARY'] = data['SUMMARY']
        df.attrs['SENSOR_VARIABLES'] = senvars
        
        Z.append(df)
        
    if len(Z) == 1:
        return Z[0]
    else:
        if verbose: print(f'Returned [{len(Z)}] stations. {[i.attrs["STID"] for i in Z]}')
        return Z
    
def networks(verbose=True, **params):
    """
    Return a DataFrame of available Networks and their metadata
    
    https://developers.synopticdata.com/mesonet/v2/networks/
    
    Parameters
    ----------
    **param : keyword arguments
       
    Some include the following
    
    id : int or list of int
        Filter by network number.
    shortname : str or list of str
        Netork shortname, i.e. 'NWS/FAA', 'RAWS', 'UTAH DOT',         
    """
    # Get the data
    web = synoptic_api('networks', verbose=verbose, **params)
    data = web.json()
    
    df = pd.DataFrame(data['MNET'])
    df.set_index('ID', inplace=True)
    df['LAST_OBSERVATION'] = pd.to_datetime(df.LAST_OBSERVATION)
    df.attrs['SUMMARY'] = data['SUMMARY']
    
    return df

def networktypes(verbose=True, **params):
    # Get the data
    web = synoptic_api('networktypes', verbose=verbose, **params)
    data = web.json()
    
    df = pd.DataFrame(data['MNETCAT'])
    df.set_index('ID', inplace=True)
    df.attrs['SUMMARY'] = data['SUMMARY']
    
    return df

def variables(verbose=True, **params):
    """
    Return a DataFrame of available station variables
    
    https://developers.synopticdata.com/mesonet/v2/variables/
    https://developers.synopticdata.com/mesonet/v2/api-variables/
    
    Parameters
    ----------
    **param : keyword arguments
       
    Some include the following
    """
    # Get the data
    web = synoptic_api('variables', verbose=verbose, **params)
    data = web.json()
    
    df = pd.concat([pd.DataFrame(i) for i in data['VARIABLES']], axis=1).transpose()
    #df.set_index('vid', inplace=True)
    df.attrs['SUMMARY'] = data['SUMMARY']
    
    return df

# Other Services
#---------------
# stations_precipitation :
# stations_nearesttime : 
# stations_latency :
# stations_latest : 
# stations_qcsegments :
# qctypes : https://developers.synopticdata.com/mesonet/v2/qctypes/



In [117]:
a = networktypes()
a


 🚚💨 Speedy Delivery from Synoptic API [networktypes]: https://api.synopticdata.com/v2//networktypes?token=2562b729557f45f5958516081f06c9eb



,PERIOD_OF_RECORD,DESCRIPTION,NAME
ID,,,
1,"{'start': '1997-01-01T00:00:00Z', 'end': '2020...",Agricultural,AG
2,"{'start': '1997-01-03T00:00:00Z', 'end': '2007...",Air Quality,AQ
3,"{'start': '1999-01-05T00:00:00Z', 'end': '2020...","Offshore, CA, MX",EXT
4,"{'start': '1997-04-01T00:00:00Z', 'end': '2019...",Federal and state networks,FED+
5,"{'start': '1997-04-01T00:00:00Z', 'end': '2014...",Hydrological,HYDRO
6,"{'start': '1998-01-27T00:00:00Z', 'end': '2004...",State and Local,LOCAL
7,"{'start': '1997-04-03T00:00:00Z', 'end': '2002...",NWS/FAA,NWS
8,"{'start': '1997-09-11T00:00:00Z', 'end': '2020...",CWOP,PUBLIC
9,"{'start': '1999-11-01T00:00:00Z', 'end': '2005...",Fire weather,RAWS


In [115]:
b = networks()
b


 🚚💨 Speedy Delivery from Synoptic API [networks]: https://api.synopticdata.com/v2//networks?token=2562b729557f45f5958516081f06c9eb



,CATEGORY,REPORTING_STATIONS,PERIOD_OF_RECORD,LAST_OBSERVATION,URL,PERCENT_REPORTING,PERIOD_CHECKED,TOTAL_STATIONS,ACTIVE_STATIONS,LONGNAME,SHORTNAME,PERCENT_ACTIVE
ID,,,,,,,,,,,,
1,7,2486,"{'start': '1997-01-01T00:00:00Z', 'end': '2020...",2020-08-12 23:15:00+00:00,None,96.81,120.0,3452,2568,National Weather Service/Federal Aviation Admi...,NWS/FAA,74.39
2,9,2279,"{'start': '1997-01-03T00:00:00Z', 'end': '2007...",2020-08-12 22:54:00+00:00,None,98.28,90.0,3373,2319,Interagency Remote Automatic Weather Stations,RAWS,68.75
3,4,28,"{'start': '1999-01-05T00:00:00Z', 'end': '2020...",2020-08-12 22:55:00+00:00,None,100.00,30.0,31,28,U.S. Army Dugway Proving Grounds,DUGWAY,90.32
4,10,136,"{'start': '1997-04-01T00:00:00Z', 'end': '2019...",2020-08-12 22:50:00+00:00,None,93.79,60.0,181,145,Utah Department of Transportation,UTAH DOT,80.11
5,4,34,"{'start': '1997-04-01T00:00:00Z', 'end': '2014...",2020-08-12 22:50:00+00:00,None,100.00,60.0,38,34,NOAA Air Resources Laboratory Field Research D...,ARL FRD,89.47
...,...,...,...,...,...,...,...,...,...,...,...,...
261,6,30,"{'start': '1997-04-12T00:00:00Z', 'end': '2020...",2020-08-12 22:50:00+00:00,None,100.00,120.0,30,30,NV Energy,NV-ENERGY,100.00
262,7,11134,"{'start': '1997-04-12T00:00:00Z', 'end': '2020...",2020-08-12 22:06:00+00:00,None,58.02,1440.0,20857,19190,"Community Collaborative Rain, Hail and Snow Ne...",COCORAHS,92.01
263,6,11,"{'start': '1997-04-12T00:00:00Z', 'end': '2020...",2020-08-12 21:58:00+00:00,None,100.00,120.0,11,11,Scripps CW3E Mesonet,CW3E,100.00


In [106]:
a

,long_name,unit,vid
air_temp,Temperature,Celsius,3
dew_point_temperature,Dew Point,Celsius,4
relative_humidity,Relative Humidity,%,5
wind_speed,Wind Speed,m/s,6
wind_direction,Wind Direction,Degrees,7
...,...,...,...
wind_cardinal_direction,Wind cardinal direction,,0
weather_summary,Weather summary,,0
heat_index,Heat index,,0
wet_bulb_temperature,Wet bulb temperature,,0


In [55]:
a = networks()


 🚚💨 Speedy Delivery from Synoptic API [networks]: https://api.synopticdata.com/v2//networks?token=2562b729557f45f5958516081f06c9eb



In [19]:
a = stations_timeseries(stid='UKBKB', vars='air_temp', recent=100)
a


 🚚💨 Speedy Delivery from Synoptic API: https://api.synopticdata.com/v2//stations/timeseries?stid=UKBKB&vars=air_temp&recent=100&token=2562b729557f45f5958516081f06c9eb



,air_temp
date_time,
2020-08-12 21:00:00+00:00,33.333
2020-08-12 21:15:00+00:00,33.333
2020-08-12 21:30:00+00:00,33.333
2020-08-12 21:45:00+00:00,33.333
2020-08-12 22:00:00+00:00,33.889
2020-08-12 22:15:00+00:00,33.333


In [268]:
a = stations_metadata(stid='wbb')
a


 🚚💨 Speedy Delivery from Synoptic API: https://api.synopticdata.com/v2//stations/metadata?stid=wbb&token=2562b729557f45f5958516081f06c9eb



,WBB
ELEVATION,4806
ELEV_DEM,4727.7
ID,1
MNET_ID,153
NAME,U of U William Browning Building
RECORD_END,2020-08-03 03:50:00+00:00
RECORD_START,1997-01-01 00:00:00+00:00
RESTRICTED,False
STATE,UT
STATUS,ACTIVE


In [254]:
a.attrs

{'URL': 'https://api.synopticdata.com/v2//stations/metadata?stid=UKBKB,WBB&token=2562b729557f45f5958516081f06c9eb',
 'elevation units': 'feet',
 'SUMMARY': {'NUMBER_OF_OBJECTS': 2,
  'RESPONSE_CODE': 1,
  'RESPONSE_MESSAGE': 'OK',
  'METADATA_RESPONSE_TIME': '1.15609169006 ms'}}

In [214]:
df.transpose()

,WBB,UKBKB
STATUS,ACTIVE,ACTIVE
MNET_ID,153,65
ELEVATION,4806,4734
NAME,U of U William Browning Building,EW2355 Spanish Fork
STID,WBB,UKBKB
ELEV_DEM,4727.7,4740.8
longitude,-111.848,-111.628
STATE,UT,UT
RESTRICTED,False,False
latitude,40.7662,40.0987


In [185]:
df['start'].apply(pd.to_datetime)

0   1997-01-01 00:00:00+00:00
1   2013-03-13 00:00:00+00:00
Name: start, dtype: datetime64[ns, UTC]

In [101]:
b = pd.DataFrame.from_dict({i for i in a['STATION']}, orient='index')
b

TypeError: unhashable type: 'dict'

In [99]:
b.loc['ELEVATION'].astype(int)

WBB    4806
Name: ELEVATION, dtype: int64

In [16]:
a = get_mesowest_stninfo(radius='ukbkb,10', obrange=[datetime(2017,9,10), datetime(2020,1,10)])
a

[datetime.datetime(2017, 9, 10, 0, 0), datetime.datetime(2020, 1, 10, 0, 0)]

🚚 Deliver data from MesoWest API: http://api.mesowest.net/v2/stations/metadata?radius=ukbkb,10&obrange=20170910,20200110&token=2562b729557f45f5958516081f06c9eb



,KPVU,QSF,C9348,FG006,FG012,D5744,UR304,UR321,UR324,UKBKB,UTPRV,UCC13,UTBRC,PUR24,COOPPROU1,COOPSPFU1,PUR76
latitude,40.2167,40.1383,40.0944,40.025,40.03,40,40.1196,40.1794,40.0404,40.0987,40.2039,40.0672,40.073,40.0847,40.24,40.08,40.0847
longitude,-111.717,-111.66,-111.631,-111.7,-111.79,-111.7,-111.679,-111.618,-111.541,-111.628,-111.655,-111.629,-111.715,-111.611,-111.65,-111.6,-111.611
NAME,"Provo, Provo Municipal Airport",Spanish Fork,CW9348 Spanish Fork,Payson,West Mountain,DW5744 Payson,ESPFRK,SPVIL1,WCSTL1,EW2355 Spanish Fork,I-15 @ Provo,Spanish Fork,I-15 @ Beer Creek,Spanish Fork Canyon,BRIGHAM YOUNG UNIV.,SPANISH FORK POWER HOUSE,Spanish Fork Canyon B
MNET_ID,1,9,65,138,138,65,64,64,64,65,4,194,4,208,79,78,208
STID,KPVU,QSF,C9348,FG006,FG012,D5744,UR304,UR321,UR324,UKBKB,UTPRV,UCC13,UTBRC,PUR24,COOPPROU1,COOPSPFU1,PUR76
ELEVATION,4498,4577,4738,4869,4683,5270,4559,4532,4907,4734,4500,4721,4529,4729,4570,4720,4707
TIMEZONE,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver,America/Denver
STATUS,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,ACTIVE,INACTIVE,ACTIVE,ACTIVE,INACTIVE
STATE,UT,UT,UT,UT,UT,UT,UT,UT,UT,UT,UT,UT,UT,UT,UT,UT,UT
RECORD_START,1997-01-01 00:00:00+00:00,2004-06-09 00:00:00+00:00,2008-01-02 00:00:00+00:00,2010-06-16 00:00:00+00:00,2010-06-16 00:00:00+00:00,2010-11-17 00:00:00+00:00,2012-11-07 00:00:00+00:00,2012-11-07 00:00:00+00:00,2012-11-07 00:00:00+00:00,2013-03-13 00:00:00+00:00,2013-03-13 00:00:00+00:00,2014-11-23 07:04:00+00:00,2014-03-12 00:00:00+00:00,2016-01-26 04:37:00+00:00,2016-06-08 01:08:00+00:00,2016-06-08 01:08:00+00:00,2016-07-26 03:07:00+00:00


In [10]:
a.attrs

{'URL': 'http://api.mesowest.net/v2/stations/metadata?radius=KLAX%2C10&token=2562b729557f45f5958516081f06c9eb'}

In [9]:
args = dict(stid='KLAX', radius='KLAX,10')
args

{'stid': 'KLAX', 'radius': 'KLAX,10'}

In [15]:
'&'.join([f'{i}={args[i]}' for i in args])

'stid=KLAX&radius=KLAX,10'

In [17]:
import requests

In [19]:
URL = 'http://api.mesowest.net/v2/stations/metadata'
params = dict(token='2562b729557f45f5958516081f06c9eb', stid='KLAX', radius='KLAX,10')

In [30]:
dir(requests.get(URL, params=params))

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [57]:
aa = requests.get(URL, params=params)
aa.url, URL

('http://api.mesowest.net/v2/stations/metadata?token=2562b729557f45f5958516081f06c9eb&stid=KLAX&radius=KLAX%2C10',
 'http://api.mesowest.net/v2/stations/metadata')

In [208]:
a = get_mesowest_stninfo(stid=['UKBKB', 'KLAX'])
a


🚚 Retrieved data from MesoWest API: http://api.mesowest.net/v2/stations/metadata?stid=UKBKB,KLAX&token=2562b729557f45f5958516081f06c9eb



,KLAX,UKBKB
latitude,33.9381,40.0987
longitude,-118.389,-111.628
NAME,"Los Angeles, Los Angeles International Airport",EW2355 Spanish Fork
MNET_ID,1,65
STID,KLAX,UKBKB
ELEVATION,125,4734
TIMEZONE,America/Los_Angeles,America/Denver
STATUS,ACTIVE,ACTIVE
STATE,CA,UT
RECORD_START,1997-04-12 00:00:00+00:00,2013-03-13 00:00:00+00:00


In [192]:
','.join([f'{i:%Y%m%d%H%M}' for i in [datetime(2017,1,1)]])

'201701010000'